# Text Analysis

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, Bidirectional 
from keras.optimizers import RMSprop

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing import sequence

from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint

from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
import pandas as pd
import numpy as np

from sklearn.utils import shuffle

import re

import gensim
#from pymystem3 import Mystem
#from stanfordcorenlp import StanfordCoreNLP
import spacy_udpipe


Using TensorFlow backend.


In [2]:
df = pd.read_csv('./task2_data/task2_en_training.tsv',delimiter='\t')
df.head()

,tweet_id,user_id,class,tweet
0,344266386467606528,809439366,0,"depression hurts, cymbalta can help"
1,349220537903489025,323112996,0,"@jessicama20045 right, but cipro can make thin..."
2,351421773079781378,713100330,0,@fibby1123 are you on paxil .. i need help
3,326594278472171520,543113070,0,@redicine the lamotrigine and sjs just made ch...
4,345567138376994816,138795534,0,have decided to skip my #humira shot today. my...


In [21]:
spacy_udpipe.download("en")
nlp = spacy_udpipe.load("en")
model = gensim.models.KeyedVectors.load_word2vec_format("./Embeddings/model.bin", binary=True) 
model.vocab 


Already downloaded a model for the 'en' language


{'say_VERB': <gensim.models.keyedvectors.Vocab at 0x7f7a0e78b050>,
 'go_VERB': <gensim.models.keyedvectors.Vocab at 0x7f7a0e78b290>,
 'make_VERB': <gensim.models.keyedvectors.Vocab at 0x7f7a0e78bb10>,
 'get_VERB': <gensim.models.keyedvectors.Vocab at 0x7f7a1ca1d050>,
 'one_NUM': <gensim.models.keyedvectors.Vocab at 0x7f7a1ca1d0d0>,
 'see_VERB': <gensim.models.keyedvectors.Vocab at 0x7f7a1ca1d190>,
 'time_NOUN': <gensim.models.keyedvectors.Vocab at 0x7f7a1ca1d250>,
 'take_VERB': <gensim.models.keyedvectors.Vocab at 0x7f7a1ca1d2d0>,
 'know_VERB': <gensim.models.keyedvectors.Vocab at 0x7f7a1ca1d350>,
 'year_NOUN': <gensim.models.keyedvectors.Vocab at 0x7f7a1ca1d390>,
 'two_NUM': <gensim.models.keyedvectors.Vocab at 0x7f7a1ca1d3d0>,
 'well_ADV': <gensim.models.keyedvectors.Vocab at 0x7f7a1ca1d450>,
 'come_VERB': <gensim.models.keyedvectors.Vocab at 0x7f7a1ca1d510>,
 'think_VERB': <gensim.models.keyedvectors.Vocab at 0x7f7a1ca1d590>,
 'give_VERB': <gensim.models.keyedvectors.Vocab at 0x7f7a

In [4]:
def lemmatize_with_postag(text = """"Wikipedia is a free online encyclopedia, created and 
                          edited by volunteers around the world.""",
                          nlp=nlp):
    doc = nlp(text)
    tagged = []
    for token in doc:
        lemma = token.lemma_
        pos = token.pos_
        
        #pos = pos.split('=')[0].strip()
        if pos not in ["PUNCT"]: #["VERB", "NUM", "ADV", "NOUN", "ADJ", "AUX", "PRON", "ADP", "DET", "ADV", "INTJ", "PROPN"]:
            tagged.append(lemma.lower() + '_' + pos)
        #else : print(pos)
    return tagged
   
print(lemmatize_with_postag())

['wikipedia_PROPN', 'be_AUX', 'a_DET', 'free_ADJ', 'online_ADJ', 'encyclopedia_NOUN', 'create_VERB', 'and_CCONJ', 'edit_VERB', 'by_ADP', 'volunteer_NOUN', 'around_ADP', 'the_DET', 'world_NOUN']


In [5]:
X_train = df['tweet']
y_train = df['class']

Xy = pd.concat([X_train, y_train], axis=1)
#print(Xy_train)

df_1 = Xy[Xy['class'] == 1].iloc[:10] # 10 твитов с индексом класса 0
df_0 = Xy[Xy['class'] == 0].iloc[:10] # 10 твитов с индексом класса 1
#print(df_0.shape)

Xy_ = shuffle(pd.concat([df_0, df_1]))
Xy_train = pd.concat([Xy_["tweet"].apply(lemmatize_with_postag), Xy_["class"]], axis = 1)
#print(Xy_train['tweet'].apply(lemmatize_with_postag).shape)

df_1 = Xy[Xy['class'] == 1].iloc[300:310]
df_0 = Xy[Xy['class'] == 0].iloc[300:310]
Xy_ = shuffle(pd.concat([df_0, df_1]))
Xy_test = pd.concat([Xy_["tweet"].apply(lemmatize_with_postag), Xy_["class"]], axis = 1)

print(Xy_train.shape)
print(Xy_test.shape)
Xy_train, Xy_test

(20, 2)
(20, 2)


(                                                 tweet  class
 499  [i_PRON, swear_VERB, thia_NOUN, cymbalta_NOUN,...      1
 532  [21.19_NUM, day_NOUN, 17_NUM, rivaroxaban_ADJ,...      1
 512  [@wickychoo_ADV, @_ADP, lollygee_NUM, omg_NOUN...      1
 4    [have_AUX, decide_VERB, to_PART, skip_VERB, my...      0
 2    [@fibby1123_NOUN, be_AUX, you_PRON, on_ADP, pa...      0
 3    [@redicine_VERB, the_DET, lamotrigine_NOUN, an...      0
 539  [@sarapekow_ADV, need_VERB, to_PART, switch_VE...      1
 529  [@znb_maqbool_INTJ, sweaty_INTJ, fluoxetine_NU...      1
 493  [try_VERB, to_PART, take_VERB, seroquel_NOUN, ...      1
 6    [#_SYM, bipolar_ADJ, meds_NOUN, think_VERB, ju...      0
 8    [everyone_PRON, be_AUX, always_ADV, upset_ADJ,...      0
 530  [i_PRON, think_VERB, baclofen_ADV, might_AUX, ...      1
 492  [@ineptune__SYM, samezy_VERB, :(_SYM, it_PRON,...      1
 494  [@flatchest_VERB, it_PRON, have_VERB, nothing_...      1
 0    [depression_NOUN, hurts_NOUN, cymbalta_NOUN, c...

In [6]:
X_train[1], lemmatize_with_postag(text = str(X_train[1]))

('@jessicama20045 right, but cipro can make things much worse...and why give bayer more of your money? they already screwed you once w/ essure',
 ['@jessicama20045_NUM',
  'right_ADJ',
  'but_CCONJ',
  'cipro_NOUN',
  'can_AUX',
  'make_VERB',
  'thing_NOUN',
  'much_ADV',
  'worse_ADJ',
  'and_CCONJ',
  'why_ADV',
  'give_VERB',
  'bayer_ADV',
  'more_ADJ',
  'of_ADP',
  'you_PRON',
  'money_NOUN',
  'they_PRON',
  'already_ADV',
  'screw_VERB',
  'you_PRON',
  'once_ADV',
  'with_ADP',
  'essure_NOUN'])

In [7]:
ttws = text_to_word_sequence(X_train[1], filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ')
ttws

['jessicama20045',
 'right',
 'but',
 'cipro',
 'can',
 'make',
 'things',
 'much',
 'worse',
 'and',
 'why',
 'give',
 'bayer',
 'more',
 'of',
 'your',
 'money',
 'they',
 'already',
 'screwed',
 'you',
 'once',
 'w',
 'essure']

In [8]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(Xy_train["tweet"]) #fit on train

# exctrax features from train
sequences_train = tok.texts_to_sequences(Xy_train["tweet"])
sequences_matrix_train = sequence.pad_sequences(sequences_train, maxlen=max_len)

# exctrax features from test
sequences_test = tok.texts_to_sequences(Xy_test["tweet"])
sequences_matrix_test = sequence.pad_sequences(sequences_test, maxlen=max_len)

In [9]:
Xy_train["tweet"][1]
sequences_matrix_train[1] 

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0, 42, 18, 43, 44, 45, 46,  2, 47,  3, 48, 19,
        7, 20, 49, 50, 51, 52, 11, 53, 54,  5, 55, 56, 57, 58],
      dtype=int32)

In [10]:
word_index = tok.word_index
print('Found %s unique tokens' % len(word_index))

word_index

Found 225 unique tokens


{'i_pron': 1,
 'be_aux': 2,
 'to_part': 3,
 'the_det': 4,
 'and_cconj': 5,
 'it_pron': 6,
 'my_pron': 7,
 'you_pron': 8,
 'of_adp': 9,
 'have_verb': 10,
 'in_adp': 11,
 'make_verb': 12,
 'a_det': 13,
 'give_verb': 14,
 'cymbalta_noun': 15,
 'need_verb': 16,
 'take_verb': 17,
 'day_noun': 18,
 'on_adp': 19,
 'body_noun': 20,
 'not_part': 21,
 'fluoxetine_noun': 22,
 'time_noun': 23,
 'just_adv': 24,
 'to_adp': 25,
 ':(_sym': 26,
 'seroquel_noun': 27,
 'all_adv': 28,
 '#_sym': 29,
 'think_verb': 30,
 'than_adp': 31,
 'with_adp': 32,
 'can_aux': 33,
 'but_cconj': 34,
 'once_adv': 35,
 'at_adv': 36,
 'swear_verb': 37,
 'thia_noun': 38,
 'cause_verb': 39,
 'horrible_adj': 40,
 'nightmare_noun': 41,
 '21.19_num': 42,
 '17_num': 43,
 'rivaroxaban_adj': 44,
 'diary_noun': 45,
 'ach_noun': 46,
 'related_adj': 47,
 'pressure_verb': 48,
 'e.g._x': 49,
 'if_sconj': 50,
 'sit_verb': 51,
 'then_adv': 52,
 'back_adv': 53,
 'buttoock_noun': 54,
 'leg_noun': 55,
 '1_num': 56,
 '/_sym': 57,
 '2_num': 58

In [24]:
def fix_word_key(string):
    upper_part = (re.findall('_.*',string))[0].upper()
    result = re.sub(r'_.*', upper_part, string)
    #print(f'was {string} -> became {result}')
    return result

for key in word_index.keys():
    fixed_key = fix_word_key(key)
    word_index[fixed_key] = word_index.pop(key)
    
print('Found %s unique tokens' % len(word_index))
word_index

Found 225 unique tokens


{'baclofen_ADV': 134,
 'might_AUX': 135,
 'bigger_ADV': 136,
 'asshole_ADJ': 137,
 'usual_ADJ': 138,
 ':/_SYM': 139,
 '@ineptune__SYM': 140,
 'samezy_VERB': 141,
 'lamictal_ADJ': 142,
 'flat_ADJ': 143,
 'as_ADP': 144,
 'kate_NOUN': 145,
 'hudson_NOUN': 146,
 'in_SCONJ': 147,
 'how_ADV': 148,
 'lose_VERB': 149,
 'guy_NOUN': 150,
 '10_NUM': 151,
 '@flatchest_VERB': 152,
 'nothing_PRON': 153,
 'do_VERB': 154,
 'any_DET': 155,
 'that_PRON': 156,
 'apparently_ADV': 157,
 'gain_VERB': 158,
 'alot_ADV': 159,
 'weight_NOUN': 160,
 'depression_NOUN': 161,
 'hurts_NOUN': 162,
 'help_VERB': 163,
 '@jessicama20045_NUM': 164,
 'right_ADJ': 165,
 'cipro_NOUN': 166,
 'thing_NOUN': 167,
 'much_ADV': 168,
 'worse_ADJ': 169,
 'why_ADV': 170,
 'bayer_ADV': 171,
 'more_ADJ': 172,
 'money_NOUN': 173,
 'they_PRON': 174,
 'already_ADV': 175,
 'screw_VERB': 176,
 'essure_NOUN': 177,
 'rt_NOUN': 178,
 '@neuronow_NOUN': 179,
 'study_NOUN': 180,
 'reinforce_NOUN': 181,
 'invokana_NOUN': 182,
 'tm_NOUN': 183,
 'c

In [31]:
# делаем Embedding на основе w2v модели

nb_words = min(max_words, len(word_index))+1 # проверяем где меньше, в нашем датасете или в токенайзере.
EMBEDDING_DIM = 300 # размерность векторов в нашей модели w2v

embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if word in model.vocab:
        print(model[word])
        embedding_matrix[i] = model[word]
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

[-2.46190e-02  7.46470e-02  4.00030e-02  8.95510e-02 -6.07880e-02
  9.82370e-02 -3.83250e-02 -4.01310e-02  2.04070e-02  8.21200e-03
 -5.34060e-02  2.90390e-02  4.00170e-02  8.30500e-03  1.51550e-02
 -6.54810e-02  4.41110e-02 -7.98830e-02 -4.70970e-02  2.00620e-02
 -3.35280e-02  4.38780e-02 -3.52830e-02  2.32000e-04 -3.13600e-02
  1.49610e-02  3.84030e-02 -3.45030e-02  4.70930e-02  7.33100e-03
  1.54500e-02 -9.85600e-03  2.29510e-02 -4.33090e-02 -2.58970e-02
 -1.13520e-02 -4.93000e-03  1.16190e-02 -5.40300e-02 -6.92660e-02
  2.32950e-02  4.76970e-02  2.40400e-03 -7.77520e-02  4.08000e-03
 -8.05670e-02  6.28180e-02  3.50380e-02  2.10680e-02  2.11430e-02
 -2.74150e-02  5.40640e-02 -2.00990e-02  1.47010e-01 -3.27260e-02
  7.68200e-02  1.36520e-02 -1.06076e-01 -2.39990e-02  1.04790e-02
  1.17110e-02  3.50430e-02  1.47578e-01  8.19010e-02  1.01390e-02
 -1.55520e-02 -3.72350e-02  1.00338e-01  9.27350e-02 -9.75570e-02
 -2.50400e-03  1.98470e-02  3.16670e-02 -6.09580e-02  9.95660e-02
  3.91860e

In [32]:
embedding_matrix.shape
embedding_matrix[135]

array([-2.46190000e-02,  7.46470019e-02,  4.00030017e-02,  8.95510018e-02,
       -6.07879981e-02,  9.82370004e-02, -3.83250006e-02, -4.01309989e-02,
        2.04070006e-02,  8.21200013e-03, -5.34060001e-02,  2.90389992e-02,
        4.00170013e-02,  8.30500014e-03,  1.51549997e-02, -6.54809996e-02,
        4.41109985e-02, -7.98830017e-02, -4.70970012e-02,  2.00619996e-02,
       -3.35280001e-02,  4.38780002e-02, -3.52829993e-02,  2.32000006e-04,
       -3.13600004e-02,  1.49609996e-02,  3.84030007e-02, -3.45030017e-02,
        4.70930003e-02,  7.33100018e-03,  1.54499998e-02, -9.85599961e-03,
        2.29509994e-02, -4.33089994e-02, -2.58970000e-02, -1.13519998e-02,
       -4.92999982e-03,  1.16189998e-02, -5.40300012e-02, -6.92659989e-02,
        2.32950002e-02,  4.76970002e-02,  2.40399991e-03, -7.77520016e-02,
        4.07999987e-03, -8.05670023e-02,  6.28179982e-02,  3.50380018e-02,
        2.10679993e-02,  2.11430006e-02, -2.74149999e-02,  5.40639982e-02,
       -2.00989991e-02,  

In [34]:
########################################
## define the model structure
########################################

rate_drop_lstm = 0.15 + np.random.rand() * 0.25
rate_drop_dense = 0.15 + np.random.rand() * 0.25
act = 'relu'


embedding_layer = Embedding(nb_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=max_len,
                            trainable=False) 

lstm_layer = Bidirectional(LSTM(64, dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm))
sequence_1_input = Input(shape=(max_len,), dtype='int32')
embedded_sequences_1 = embedding_layer(sequence_1_input)
x1 = lstm_layer(embedded_sequences_1)
merged = Dropout(rate_drop_dense)(x1)
merged = BatchNormalization()(merged)

merged = Dense(32, activation=act)(merged)
merged = Dropout(rate_drop_dense)(merged)
merged = BatchNormalization()(merged)

preds = Dense(1, activation='sigmoid')(merged)

In [35]:
########################################
## train the model
########################################
STAMP = 'lstm_%d_%d_%.2f_%.2f'%(10, 10, rate_drop_lstm, \
        rate_drop_dense)

model = Model(inputs=[sequence_1_input], \
        outputs=preds)
model.compile(loss='binary_crossentropy',
        optimizer='nadam',
        metrics=['acc'])
#model.summary()
#print(STAMP)

early_stopping =EarlyStopping(monitor='val_loss', patience=10)
bst_model_path = STAMP + '.h5'
model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)

hist = model.fit(sequences_matrix_train, Xy_train["class"], \
        validation_data=(sequences_matrix_test, Xy_test["class"]), \
        epochs=200, batch_size=100, shuffle=True, \
        callbacks=[early_stopping, model_checkpoint])

Train on 20 samples, validate on 20 samples
Epoch 1/200
20/20 [==============================] - 1s 73ms/step - loss: 0.8626 - acc: 0.6500 - val_loss: 0.6921 - val_acc: 0.5000
Epoch 2/200
20/20 [==============================] - 0s 5ms/step - loss: 0.7237 - acc: 0.6000 - val_loss: 0.6920 - val_acc: 0.5000
Epoch 3/200
20/20 [==============================] - 0s 5ms/step - loss: 0.9283 - acc: 0.6000 - val_loss: 0.6920 - val_acc: 0.5000
Epoch 4/200
20/20 [==============================] - 0s 5ms/step - loss: 0.5352 - acc: 0.7500 - val_loss: 0.6926 - val_acc: 0.5000
Epoch 5/200
20/20 [==============================] - 0s 5ms/step - loss: 0.6641 - acc: 0.6000 - val_loss: 0.6928 - val_acc: 0.5000
Epoch 6/200
20/20 [==============================] - 0s 5ms/step - loss: 0.7737 - acc: 0.5000 - val_loss: 0.6924 - val_acc: 0.5000
Epoch 7/200
20/20 [==============================] - 0s 5ms/step - loss: 0.5274 - acc: 0.7000 - val_loss: 0.6923 - val_acc: 0.5000
Epoch 8/200
20/20 [===================

In [43]:
pred = model.predict(sequences_matrix_test)
pred.shape
print(pred)

[[0.52457803]
 [0.52197313]
 [0.530939  ]
 [0.5158763 ]
 [0.51736104]
 [0.51519495]
 [0.5135038 ]
 [0.52457803]
 [0.51067543]
 [0.52457803]
 [0.52237886]
 [0.53659075]
 [0.5303249 ]
 [0.52385813]
 [0.5233281 ]
 [0.52457803]
 [0.52549285]
 [0.52457803]
 [0.52457803]
 [0.52457803]]


In [37]:
from sklearn.metrics import classification_report
print(classification_report(Xy_test['class'], pred))

(20, 1)